### Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import os
import random

from commons import myfunctions as myfunc


#### Carpeta ejecuciones/parámetros

Buscar fichero "02_parametros.csv" y genera ficheros yaml para utilizar con _papermill_ en la carpeta _parametros_

El fichero CSV debe tener como mínimo las siguientes columnas:

* num:      indice, no se utiliza.
* orden:    si se quiere que se lancen en ese orden los notebooks (opcional)
* M_TIPO:   identificador del tipo de conjunto de datos
* M_SELECT: identificador del algoritmo de seleccion de características utilizado
* M_CLASIF: identificador del algoritmo de clasificiación utilizado
* eval_selector:    función scikit-learn a utilizar del algoritmo de selección y que será el argumento de la función eval()
* eval_clasifier: función scikit-learn a utilizar del algirtmo de clasificación y que será el argumento de la función eval()


El fichero YAML generado con los parámetros, además de las columnas anteriores tendrá:
* P_FICHERO: nombre del fichero de parámetros
* R_FICHERO: nombre del fichero donde dejará los resultados de la valoraciones del RandomSearchCV
* M_FICHERO: nombre del fichero donde dejará las métricas
* DATA_FILE: nombre del fichero con el conjunto de datos. Se concatenará con M_TIPO.
* SCORING:   función de evaluación para el RandomSearchCV
* MAX_ITERS: número máximo de iteraciones del RandomSearchCV a ejecutar

Un ejemplo de las dos primeras líneas del fichero CSV:

<pre>
num;orden;M_TIPO;M_SELECT;M_CLASIF;eval_selector;eval_clasifier;param_distributions
0;0;bin_m;ANOVA;SVM;SelectKBest(score_func=f_classif);SVC(probability=True, random_state=SEMILLA);{'selector__k': [500],  'clasifier__kernel': ['rbf'], 'clasifier__gamma': [0.01], 'clasifier__C': [100]}
</pre>

In [ ]:
# Inicializar semilla para el random
np.random.seed(42)

carpeta_origen = "ejecuciones"

carpetah=os.path.join(carpeta_origen)
if not os.path.exists(carpetah):
    os.mkdir(carpetah)

carpetah=os.path.join(carpeta_origen,"training")
if not os.path.exists(carpetah):
    os.mkdir(carpetah)

carpetah=os.path.join(carpeta_origen,"metricas")
if not os.path.exists(carpetah):
    os.mkdir(carpetah)

carpetah=os.path.join(carpeta_origen,"modelos")
if not os.path.exists(carpetah):
    os.mkdir(carpetah)

carpetah=os.path.join(carpeta_origen,"parametros")
if not os.path.exists(carpetah):
    os.mkdir(carpetah)
    os.mkdir(os.path.join(carpetah,"error"))
    os.mkdir(os.path.join(carpetah,"done"))
else:
    print("Carpeta",carpetah,"ya existe.")

df1 = myfunc.read_csv_to_df_spa("02_parametros_definitivos.csv", os.path.join("."))

display(df1.sample(3))



Buscar semillas a utilizar

In [ ]:
# Generar 10 valores para las semillas
SEMILLAS = np.random.randint(1000, 9999, size=10)

# Si solo se quiere la semilla 42
SEMILLAS = [42]

print(SEMILLAS)


Generar los ficheros

In [ ]:

print("Generando ficheros en ", carpetah)
print("Semillas:", end=" ")
for SEMILLA in SEMILLAS:
  print(SEMILLA, end=", ")

  for i in range(len(df1)):
    datos = df1.iloc[i]

    if "orden" not in df1.columns:
      orden=0
    else:
      orden=datos.orden

    P_FICHERO = "param_"+str(orden).zfill(4)+"_"+str(SEMILLA).zfill(4)+"_"+datos.M_TIPO+"_"+datos.M_SELECT+"_"+datos.M_CLASIF+".yaml"
    R_FICHERO = "resultados_"+str(SEMILLA).zfill(4)
    M_FICHERO = "metricas_"+str(SEMILLA).zfill(4)
    DATA_FILE = "rows_transpose_norm_by_gene_id_with_target_num_"
    SCORING = "roc_auc_ovr"
    MAX_ITERS = 500

    fichero1=os.path.join(carpetah,P_FICHERO)
    with open(fichero1, "w") as f:
      f.write(f'M_TIPO: "{datos.M_TIPO}"\n')
      f.write(f'M_SELECT: "{datos.M_SELECT}"\n')
      f.write(f'M_CLASIF: "{datos.M_CLASIF}"\n')
      f.write(f'P_FICHERO: "{P_FICHERO}"\n')    
      f.write(f'R_FICHERO: "{R_FICHERO}"\n')
      f.write(f'M_FICHERO: "{M_FICHERO}"\n')
      f.write(f'PRE_DATA_FILE: "{DATA_FILE}"\n')
      f.write(f'SCORING: "{SCORING}"\n')
      f.write(f'SEMILLA: {str(SEMILLA)}\n')
      f.write(f'MAX_ITERS: {(MAX_ITERS)}\n')
      f.write(f'eval_selector: "{datos.eval_selector}"\n')
      f.write(f'eval_clasifier: "{datos.eval_clasifier}"\n')
      f.write(f'param_distributions: {datos.param_distributions}\n')
